# 텐서플로를 사용하는 방법(tf.Tensor)

- `tf.constant([[1,2,3],[4,5,6]])`
- 계산
  - `tf.square(t)` 
  - `t@tf.transpose(t)` 
  - `tf.add(t,t), tf.multiply(t,t), tf.square(t), tf.exp(t), tf.sqrt(t)`
  - `tf.reduce_sum(t), tf.reduce_mean(t), tf.reduce_max(t), tf.math.log(t)`
- 형식변경
  - `tf.constant(a), t.numpy(), np.array(t), tf.square(a), np.square(t)`
- 타입 변경
  - `tf.constant(40., dtype=tf.float64)`
- 문자열
```
tf.constant(b"hello")
tf.constant("hello")
enc=tf.strings.unicode_encode(tf.constant([ord(c) for c in "hello"]),"UTF-8")
dec=tf.strings.unicode_decode(enc, "UTF-8")
```

- 희소 텐서 `tf.SparseTensor()`
- to_dense `tf.sparse.to_dense(t)` : 희소 텐서를 실제 텐서 형태로 리턴해줌 (0 채워서)
```
t = tf.SparseTensor(indices=[[0, 1], [1, 0], [2, 3]],
                    values=[1., 2., 3.],
                    dense_shape=[3, 4])
print(t)
print(tf.sparse.to_dense(t))
```
- 집합 `tf.sets.union(tensor1,tensor2)`, `tf.sets.intersection(tensor1,tensor2)`, `tf.sets.difference(tensor1,tensor2)`
  
# 인덱싱 하는법
- `...`와 `:`는 같은 효과이다.
```
t=tf.constant([[1,2,3],[4,5,6]])

print(t[:,1:])
print(t[...,1:])

```
- 인덱싱할 때 `tf.newaxis` 넣으면 1차원을 늘릴수가 있다. 
```
t=tf.constant([[1,2,3],[4,5,6]])

t[:,:].shape, t[:,:,tf.newaxis].shape #(2,3) -> (2,3,1)
t[:,1].shape, t[:,1,tf.newaxis].shape #(2) -> (2,1)
t[1,:].shape, t[tf.newaxis,1,:].shape #(3) -> (1,3)
```

# tf.Variable
- 변수에 해당하는 Variable은 변숫값을 함부러 바꾸지 못하고, v.assign을 통해 바꿀 수 있다. 
```
v = tf.Variable([[1., 2., 3.], [4., 5., 6.]])
v.assign(2*v)
```
- 그 외 `v.assign_add()`, `v.assign_sub()`


In [1]:
import tensorflow as tf

t=tf.constant([[1,2,3],[4,5,6]])


In [2]:
t[:,1:],t[...,1:]

(<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
 array([[2, 3],
        [5, 6]])>,
 <tf.Tensor: shape=(2, 2), dtype=int32, numpy=
 array([[2, 3],
        [5, 6]])>)

In [15]:
t=tf.constant([[1,2,3],[4,5,6]])
new=t[:,:,tf.newaxis] #
t[:,:].shape, t[:,:,tf.newaxis].shape #(2,3) -> (2,3,1)
t[:,1].shape, t[:,1,tf.newaxis].shape #(2) -> (2,1)
t[1,:].shape, t[tf.newaxis,1,:].shape #(3) -> (1,3)

(TensorShape([3]), TensorShape([1, 3]))

# 공통  
- `__init__(self, **kwargs)`
- `get_config(self)`   
# 사용자 정의 손실 함수  
- `keras.losses.Loss`를 상속받으며, `call(self,y_true,y_pred)`에서 새로운 loss 결과를 리턴한다.  
- `model.compile(loss=Loss클래스(), optimizer=..., metrics=[...])`    
# 사용자 정의 활성화함수(activation) / 초기화(initializer) / 규제(regularizer) / 제한(constraint)   
- `model.layers.Dense(안에, activation='', kernel_regularizer='', kernel_initializer='', kernel_constraint='')
- initializer함수: shape를 받아서 초기화하는 함수
- regularizer함수: weights을 받아서, 원래의 Loss에 더해지는 값
- constraint함수: weights를 받아서, weights와 같은 차원을 값으로 변형해서 리턴해주는 함수  
# 사용자 정의 metric 함수  
- `keras.metrics.Metric`을 상속받으며, `update_state(self,y_true,y_pred)` -> `result(self)`순서로 호출
- result에 새로운 metric 결과를 리턴한다
- `model.complie(loss=..., optimizer=..., weighted_metrics=[Metric클래스()])`  
# 사용자 지정 레이어
- `keras.layers.Layer`를 상속 받음
- `build()`: 가중치마다 add_weights() 메서드를 호출하여 층의 변수를 만든다. 층이 처음 사용될 때 호출된다. 필수 메서드는 아니다.
- `call()`: 층에 필요한 연산을 수행한다. 활성화 함수까지 들어간다. 이 값이 층의 출력에 해당한다. 
- `call()`안에 `training=None` 옵션을 넣으면 학습과 검증을 다르게 할 수 있다. 